In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures

In [2]:
# read all the words

words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# Build a vocabulary of characters and mapping to/from integers

chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}

In [73]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      # print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182437, 3]) torch.Size([182437])
torch.Size([22781, 3]) torch.Size([22781])
torch.Size([22928, 3]) torch.Size([22928])


In [74]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g) # embedding matrix that represents each character as a 2 dimensional array
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [75]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

In [76]:
for p in parameters:
    p.requires_grad = True

In [77]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10 ** lre

In [80]:
lri = []
lossi = []

for i in range(10000):
    
    # mini batches; its faster to have an approximate gradient and take more steps than have an exact one with fewer steps
    ix = torch.randint(0, Xtr.shape[0], (32,))

    # forward pass
    emb = C[Xtr[ix]] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2 # (32, 27)
    loss = F.cross_entropy(logits, Ytr[ix])
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # Update
    # lr = lrs[i]
    lr = 0.01
    for p in parameters:
        p.data += -lr * p.grad
        
    # track stats
    # lri.append (lre[i])
    # lossi.append(loss.item())

print(loss.item())

2.0643515586853027


In [81]:
# loss for the whole dataset

emb = C[Xdev]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.4076, grad_fn=<NllLossBackward0>)

### Why PyTorches Cross Entropy function is better than doing it manually

The cross entropy function is way more efficient as it bundles up all the mathematical operations into one kernel and doesnt use any extra memory to create intermediate tensors. The forward and backward pass is also more straightfoward. one other important advantage is that when you take the exp of a number, you can run out of bounds giving inf. Pytorch cross entropy function solves this by negating the highest valued number from the logits as that doesn't change the probability distribution

### Why loss reduces super quick but doesn't go to zero

Another thing, the loss here is super low. This is because we have a lot of parameters and only 32 training examples. The model is unable however to have zero loss as ... input is followed by 4 different letters in all the 4 names so it isn't able to learn that. Right now it is learning all the unique inputs and outputs, thus not generalizing well

### How to determine the right learning rate?

First choose a learning rate and see how the loss performs, does it explode, does it not move. Decide a suitable range and create an array. plot this array against the loss and choose the learning rate with the lowest loss.